# ⚡ Day 3: Production ML at Scale - Spark, Kubernetes & Graph DBs

**🎯 Goal:** Master enterprise-scale ML infrastructure & specialized databases

**⏱️ Time:** 150-180 minutes

**🌟 Why This Matters (2025 Industry Reality):**
- **Spark/Big Data** appears in 15% of ML job postings - companies have massive datasets
- **Kubernetes** is THE standard for deploying ML models at scale
- **Graph Databases** power knowledge graphs for advanced RAG systems
- These skills separate "toy projects" from "production ML engineers"
- Every FAANG company uses these technologies

**What You'll Master Today:**
1. **PySpark for ML** - Process billions of rows, train distributed models
2. **Kubernetes for ML** - Deploy, scale, and manage ML services
3. **Graph Databases** - Build knowledge graphs with Neo4j
4. **Production Patterns** - What Fortune 500 companies actually use

---

## 🔥 Part 1: Big Data ML with PySpark

**The Problem:**
```python
# This fails with 1 billion rows:
df = pd.read_csv("massive_dataset.csv")  # ❌ Out of memory!
```

**The Solution: Apache Spark**
- Distributed computing framework
- Process data across multiple machines
- Handle petabytes of data
- Built-in ML library (MLlib)

### 🎯 PySpark Use Cases:
- Data preprocessing for large datasets
- Feature engineering at scale
- Training models on distributed data
- Real-time stream processing

### 📊 When to Use Spark:

| Data Size | Tool | Why |
|-----------|------|-----|
| < 10 GB | Pandas | Fits in memory |
| 10-100 GB | Dask/Polars | Out-of-core processing |
| > 100 GB | **Spark** | Distributed processing |
| Streaming | **Spark Streaming** | Real-time ML |

Let's learn PySpark!

In [ ]:
# Install PySpark
import sys
!{sys.executable} -m pip install pyspark findspark --quiet

print("✅ PySpark installed!")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
import pandas as pd
import numpy as np

# Create Spark session
spark = SparkSession.builder \
    .appName("MLAtScale") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ Spark Session Created!")
print(f"   Spark Version: {spark.version}")
print(f"   App Name: {spark.sparkContext.appName}")

# Show Spark UI (in local mode)
print(f"\n💡 Spark UI: http://localhost:4040")

In [ ]:
# Create large-scale training dataset (simulating real production data)
# In production, this would be billions of rows from data warehouses

np.random.seed(42)
n_samples = 100000  # 100K rows (imagine this is 100M in production)

data = pd.DataFrame({
    'user_id': range(n_samples),
    'age': np.random.randint(18, 70, n_samples),
    'income': np.random.randint(20000, 200000, n_samples),
    'num_purchases': np.random.randint(0, 50, n_samples),
    'time_on_site': np.random.randint(1, 3600, n_samples),  # seconds
    'num_clicks': np.random.randint(0, 100, n_samples),
})

# Create target: high-value customer (will buy premium product)
data['will_convert'] = (
    (data['income'] > 100000) & 
    (data['num_purchases'] > 20) &
    (data['time_on_site'] > 1000)
).astype(int)

# Convert to Spark DataFrame
spark_df = spark.createDataFrame(data)

print("✅ Large-scale dataset created!")
print(f"   Rows: {spark_df.count():,}")
print(f"   Columns: {len(spark_df.columns)}")
print("\nSchema:")
spark_df.printSchema()

In [ ]:
# Data Exploration at Scale with Spark
print("📊 Data Exploration with Spark:\n")
print("="*80)

# Show first rows
print("\nFirst 5 rows:")
spark_df.show(5)

# Aggregations (computed across distributed data)
print("\nAggregations:")
spark_df.select(
    avg("age").alias("avg_age"),
    avg("income").alias("avg_income"),
    avg("num_purchases").alias("avg_purchases")
).show()

# Group by (distributed group-by across cluster)
print("\nConversion Rate by Age Group:")
spark_df.withColumn(
    "age_group",
    when(col("age") < 30, "18-29")
    .when(col("age") < 50, "30-49")
    .otherwise("50+")
).groupBy("age_group").agg(
    count("*").alias("count"),
    avg("will_convert").alias("conversion_rate")
).show()

print("💡 These operations work on billions of rows across clusters!")

In [ ]:
# Machine Learning with Spark MLlib
print("🤖 Training ML Model with PySpark MLlib\n")
print("="*80)

# Prepare features
feature_columns = ['age', 'income', 'num_purchases', 'time_on_site', 'num_clicks']

# Create ML Pipeline
assembler = VectorAssembler(
    inputCols=feature_columns,
    outputCol="raw_features"
)

scaler = StandardScaler(
    inputCol="raw_features",
    outputCol="features"
)

lr = LogisticRegression(
    labelCol="will_convert",
    featuresCol="features",
    maxIter=10
)

# Build pipeline
pipeline = Pipeline(stages=[assembler, scaler, lr])

# Split data
train_df, test_df = spark_df.randomSplit([0.8, 0.2], seed=42)

print(f"Training set: {train_df.count():,} rows")
print(f"Test set: {test_df.count():,} rows")

# Train model (distributed training!)
print("\n🚀 Training distributed model...")
model = pipeline.fit(train_df)
print("✅ Model trained!")

# Evaluate
predictions = model.transform(test_df)

# Show predictions
print("\n📊 Sample Predictions:")
predictions.select(
    "age", "income", "num_purchases", 
    "will_convert", "prediction", "probability"
).show(10, truncate=False)

# Calculate accuracy
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(
    labelCol="will_convert",
    metricName="areaUnderROC"
)

auc = evaluator.evaluate(predictions)
print(f"\n📈 Model Performance:")
print(f"   AUC-ROC: {auc:.4f}")

print("\n💡 This same code works on billions of rows across Spark clusters!")

### 🎯 PySpark Best Practices for ML:

**1. Lazy Evaluation:**
```python
# Spark doesn't execute until you call an action
df = df.filter(col("age") > 25)  # Not executed yet
df = df.select("age", "income")  # Still not executed
df.show()  # NOW it executes (optimized query plan)
```

**2. Partitioning:**
```python
# Partition data for parallel processing
df = df.repartition(100)  # 100 partitions across cluster
```

**3. Caching:**
```python
# Cache frequently-used DataFrames in memory
df.cache()
```

**4. Avoid UDFs (User-Defined Functions):**
```python
# ❌ Slow (Python UDF)
from pyspark.sql.functions import udf
my_udf = udf(lambda x: x * 2)

# ✅ Fast (built-in Spark function)
df.withColumn("doubled", col("value") * 2)
```

## ☸️ Part 2: Kubernetes for ML Deployment

**What is Kubernetes (K8s)?**

Kubernetes is a container orchestration platform that manages:
- **Deployment**: Roll out models as services
- **Scaling**: Auto-scale based on load
- **Load Balancing**: Distribute traffic across replicas
- **Self-Healing**: Restart failed containers
- **Rolling Updates**: Update models with zero downtime

### 🎯 Why Kubernetes for ML?

**Traditional Deployment:**
```
Single server → Model crashes → Everything down ❌
```

**Kubernetes Deployment:**
```
Multiple replicas → One crashes → Others handle traffic ✅
High traffic → Auto-scale to 10 replicas ✅
New model → Rolling update, zero downtime ✅
```

### 📊 Kubernetes Architecture for ML:

```
┌─────────────────────────────────────────────┐
│           Kubernetes Cluster                │
│                                             │
│  ┌─────────────┐  ┌─────────────┐         │
│  │   Pod 1     │  │   Pod 2     │         │
│  │  (ML Model) │  │  (ML Model) │  ...    │
│  └─────────────┘  └─────────────┘         │
│         ↑                ↑                  │
│         └────────────────┘                 │
│                │                            │
│         ┌──────────────┐                   │
│         │    Service   │ ← Load Balancer   │
│         └──────────────┘                   │
└─────────────────────────────────────────────┘
                 ↑
            User Requests
```

### 🐳 Deploying ML Model to Kubernetes

**Step 1: Containerize Model (Docker)**

```dockerfile
# Dockerfile
FROM python:3.9-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install -r requirements.txt

# Copy model and code
COPY model.pkl .
COPY app.py .

# Expose port
EXPOSE 8000

# Run FastAPI server
CMD ["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]
```

**Step 2: Create Kubernetes Deployment**

```yaml
# deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: ml-model-deployment
spec:
  replicas: 3  # Run 3 copies of the model
  selector:
    matchLabels:
      app: ml-model
  template:
    metadata:
      labels:
        app: ml-model
    spec:
      containers:
      - name: ml-model
        image: your-registry/ml-model:v1.0
        ports:
        - containerPort: 8000
        resources:
          requests:
            memory: "1Gi"
            cpu: "500m"
          limits:
            memory: "2Gi"
            cpu: "1000m"
        # Health checks
        livenessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
```

**Step 3: Create Service (Load Balancer)**

```yaml
# service.yaml
apiVersion: v1
kind: Service
metadata:
  name: ml-model-service
spec:
  type: LoadBalancer
  selector:
    app: ml-model
  ports:
  - protocol: TCP
    port: 80
    targetPort: 8000
```

**Step 4: Auto-Scaling Configuration**

```yaml
# autoscaler.yaml
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: ml-model-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: ml-model-deployment
  minReplicas: 2
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 70  # Scale when CPU > 70%
```

**Deploy to Kubernetes:**

```bash
# Build and push Docker image
docker build -t your-registry/ml-model:v1.0 .
docker push your-registry/ml-model:v1.0

# Deploy to Kubernetes
kubectl apply -f deployment.yaml
kubectl apply -f service.yaml
kubectl apply -f autoscaler.yaml

# Check status
kubectl get pods
kubectl get services
kubectl get hpa

# View logs
kubectl logs -f deployment/ml-model-deployment

# Update model (rolling update)
kubectl set image deployment/ml-model-deployment ml-model=your-registry/ml-model:v2.0
```

### 🎯 Kubernetes for ML: Best Practices

**1. Resource Management:**
```yaml
resources:
  requests:  # Minimum guaranteed
    memory: "1Gi"
    cpu: "500m"
  limits:  # Maximum allowed
    memory: "2Gi"
    cpu: "1000m"
```

**2. Health Checks:**
- **Liveness Probe**: Restart if unhealthy
- **Readiness Probe**: Don't send traffic if not ready

**3. Model Versioning:**
```yaml
# Use image tags for versioning
image: ml-model:v1.0  # Not :latest!
```

**4. Secrets Management:**
```yaml
# Store API keys in Kubernetes secrets
env:
- name: OPENAI_API_KEY
  valueFrom:
    secretKeyRef:
      name: api-secrets
      key: openai-key
```

**5. GPU Support:**
```yaml
resources:
  limits:
    nvidia.com/gpu: 1  # Request 1 GPU
```

## 🕸️ Part 3: Graph Databases & Knowledge Graphs

**What are Graph Databases?**

Traditional databases store data in tables:```
Users Table | Products Table
```

Graph databases store relationships:
```
(User)-[PURCHASED]->(Product)
(User)-[FRIENDS_WITH]->(User)
(Product)-[SIMILAR_TO]->(Product)
```

### 🎯 Why Graph DBs for AI?

**Use Cases:**
1. **Knowledge Graphs for RAG** - Connect entities for better retrieval
2. **Recommendation Systems** - Find similar items through relationships
3. **Fraud Detection** - Detect suspicious patterns in transactions
4. **Entity Resolution** - Link related entities across datasets
5. **Semantic Search** - Navigate knowledge through relationships

### 🌟 Neo4j: Leading Graph Database

**Example: AI Knowledge Graph**
```cypher
(GPT-4)-[:IS_A]->(LLM)
(GPT-4)-[:USES]->(Transformer)
(GPT-4)-[:TRAINED_BY]->(OpenAI)
(Transformer)-[:INVENTED_IN]->(2017)
(Transformer)-[:USES]->(Self-Attention)
```

In [ ]:
# Install Neo4j driver
!{sys.executable} -m pip install neo4j --quiet

print("✅ Neo4j driver installed!")
print("\n💡 To use Neo4j:")
print("   1. Install Neo4j Desktop or use Neo4j Aura (cloud)")
print("   2. Create a database")
print("   3. Get connection URI and credentials")
print("\n   Neo4j Aura (free): https://neo4j.com/cloud/aura/")

In [ ]:
# Example: Building an AI Knowledge Graph
# (Demo code - requires Neo4j instance)

from neo4j import GraphDatabase

# Example connection (uncomment and add your credentials)
'''
# Connect to Neo4j
driver = GraphDatabase.driver(
    "neo4j+s://your-instance.databases.neo4j.io",
    auth=("neo4j", "your-password")
)

def create_ai_knowledge_graph(tx):
    # Create nodes and relationships
    query = """
    // Create AI concepts
    CREATE (gpt:Model {name: "GPT-4", params: "1.8T"})
    CREATE (claude:Model {name: "Claude 3", params: "Unknown"})
    CREATE (llm:Category {name: "LLM"})
    CREATE (transformer:Architecture {name: "Transformer"})
    CREATE (attention:Mechanism {name: "Self-Attention"})
    CREATE (rag:Technique {name: "RAG"})
    CREATE (lora:Technique {name: "LoRA"})
    CREATE (openai:Company {name: "OpenAI"})
    CREATE (anthropic:Company {name: "Anthropic"})
    
    // Create relationships
    CREATE (gpt)-[:IS_A]->(llm)
    CREATE (claude)-[:IS_A]->(llm)
    CREATE (gpt)-[:USES]->(transformer)
    CREATE (claude)-[:USES]->(transformer)
    CREATE (transformer)-[:USES]->(attention)
    CREATE (gpt)-[:DEVELOPED_BY]->(openai)
    CREATE (claude)-[:DEVELOPED_BY]->(anthropic)
    CREATE (rag)-[:IMPROVES]->(llm)
    CREATE (lora)-[:FINE_TUNES]->(llm)
    """
    tx.run(query)

# Create the graph
with driver.session() as session:
    session.execute_write(create_ai_knowledge_graph)
    
print("✅ AI Knowledge Graph created!")
'''

print("👆 Uncomment and add Neo4j credentials to run!")
print("\n💡 Once created, you can query the graph:")

### 📊 Querying Knowledge Graphs (Cypher)

**Example Queries:**

**1. Find all LLMs:**
```cypher
MATCH (m:Model)-[:IS_A]->(llm:Category {name: "LLM"})
RETURN m.name, m.params
```

**2. Find what techniques improve LLMs:**
```cypher
MATCH (t:Technique)-[:IMPROVES]->(llm:Category {name: "LLM"})
RETURN t.name
```

**3. Find all models by OpenAI:**
```cypher
MATCH (m:Model)-[:DEVELOPED_BY]->(c:Company {name: "OpenAI"})
RETURN m.name
```

**4. Find shortest path between two concepts:**
```cypher
MATCH path = shortestPath(
  (gpt:Model {name: "GPT-4"})-[*]-(attn:Mechanism {name: "Self-Attention"})
)
RETURN path
```

**5. Recommend similar models:**
```cypher
MATCH (m1:Model {name: "GPT-4"})-[:USES]->(arch:Architecture)<-[:USES]-(m2:Model)
WHERE m1 <> m2
RETURN m2.name as similar_model
```

### 🎯 Knowledge Graphs for RAG

**Traditional RAG:**
```
Query → Vector Search → Documents → LLM → Answer
```

**Graph-Enhanced RAG:**
```
Query → Vector Search → Documents
         ↓
    Extract Entities → Graph Traversal → Related Entities
         ↓
    Enriched Context → LLM → Better Answer with Relationships
```

**Example:**

**Query:** "How does GPT-4 work?"

**Traditional RAG:**
- Retrieves: "GPT-4 is a language model"

**Graph RAG:**
- Retrieves: "GPT-4 is a language model"
- Graph traversal finds:
  - GPT-4 uses Transformers
  - Transformers use Self-Attention
  - Developed by OpenAI
  - Related to Claude 3
- LLM gets richer context → Better answer!

## 🎓 Key Takeaways

### PySpark for Big Data ML:
✅ **Distributed Computing** - Process billions of rows across clusters  
✅ **MLlib** - Distributed machine learning algorithms  
✅ **15% of jobs require Spark** - Essential for large-scale ML  
✅ **Use when:** Data > 100GB, need distributed training  

### Kubernetes for ML:
✅ **Container Orchestration** - Deploy, scale, manage ML services  
✅ **Auto-Scaling** - Handle variable traffic automatically  
✅ **Zero-Downtime Updates** - Rolling deployments for new models  
✅ **Industry Standard** - Used by all major tech companies  

### Graph Databases:
✅ **Relationship-Focused** - Model connections between entities  
✅ **Knowledge Graphs** - Power advanced RAG systems  
✅ **Neo4j** - Leading graph database with Cypher query language  
✅ **Use Cases** - Recommendations, fraud detection, semantic search  

---

## 🏆 Week 21 Complete!

**You've mastered industry-critical skills:**

### Day 1:
- 🗄️ SQL for AI/ML data workflows
- 🌲 Pinecone for production RAG
- 🔷 Weaviate for hybrid search
- 🎨 ChromaDB for local development

### Day 2:
- 📚 Advanced RAG (chunking, hybrid search, re-ranking)
- 🤖 LLM Agents (ReAct, tool use, function calling)
- 🎨 LoRA/QLoRA (parameter-efficient fine-tuning)

### Day 3:
- 🔥 PySpark for big data ML
- ☸️ Kubernetes for ML deployment at scale
- 🕸️ Graph databases for knowledge graphs

---

## 🚀 You're Now Industry-Ready!

**Skills Gap Closed:**
✅ SQL (26% of jobs) ✓  
✅ Vector Databases (Top 2025 skill) ✓  
✅ Advanced RAG (Most in-demand) ✓  
✅ LLM Agents (Emerging trend) ✓  
✅ LoRA/QLoRA (21% of jobs) ✓  
✅ Spark (15% of jobs) ✓  
✅ Kubernetes (Production standard) ✓  
✅ Graph Databases (Advanced RAG) ✓  

**What This Means:**
- You can build production RAG systems
- You can deploy models at scale
- You can process big data
- You understand what companies actually use
- You're competitive for senior ML roles

**💼 You're ready for:**
- ML Engineer positions at FAANG
- AI Engineer roles at startups
- MLOps Engineer positions
- Senior Data Scientist roles

---

**🎉 Congratulations on completing the Learn-AI curriculum!**

**From Week 1 Python basics to Week 21 production ML at scale - you've covered everything needed to become an AI engineer in 2025!**

**Now go build amazing AI applications! 🚀**